# Variational Quantum Eigensolver VQE
It is a typical quantum-classical hybrid algorithm that runs on modern quantum computers and uses both quantum and classical computers.

## What we learn
1. Learn the basic theory of VQE
2. Implementing Quantum Variational Circuits in Blueqat

## Overview
In addition to a universal algorithm that assumes an ideal error-correcting quantum computer, there is a variational algorithm for the NISQ of current quantum computers.

1. Universal (Grover, Shor, Phase Estimation, Quantum Fourier Transform, HHL, Quantum Support Vector Machine, etc.)
2. variational (VQE,QAOA)

VQE is an algorithm that uses short quantum circuits to solve problems in a hybrid way with existing computers.

## Eigenvalue problem
VQE calculates the expectation value of the eigenvalue of a given matrix (Hamiltonian). Various problems can be solved by solving eigenvalue problems. If the eigenvalue is $E_0$ and the eigenvector is $\mid \psi \rangle, the expectation value is,

$$
H \mid \psi \rangle = E_0 \mid \psi \rangle
$$

The purpose is to find $E_0$.

## Hamiltonian and expectations

The problem is passed in an Hermitian matrix called the Hamiltonian $H$. The Hamiltonian consists of a Pauli matrix and a unit matrix, which takes the form of a complex matrix.

In [1]:
from blueqat.pauli import X, Y, Z, I

h = 1.23 * I - 4.56 * X(0) + 2.45 * Y(0) + 2.34 * Z(0)
h.to_matrix()

array([[ 3.57+0.j  , -4.56-2.45j],
       [-4.56+2.45j, -1.11+0.j  ]])

Expectation value of Hamiltonian is,

$$
\langle \psi \mid H \mid \psi \rangle
$$

And, the Hamiltonian expectation can be decomposed because it is a linear combination of unitary matrices.

$$
\langle \psi \mid aH_1 + bH_2 \mid \psi \rangle \\ = \langle \psi \mid aH_1 \mid \psi \rangle + \langle \psi \mid bH_2 \mid \psi \rangle \\ = a\langle \psi \mid H_1 \mid \psi \rangle + b\langle \psi \mid H_2 \mid \psi \rangle
$$

For example,

$$
H = 1.2 X_0 Z_2 + 2.5 Z_0 X_1 Y_2 - 3.4 Z_2 X_1
$$

The expected value of this formula can be obtained as follows.

$$
\langle \psi \mid 1.2 X_0 Z_2 + 2.5 Z_0 X_1 Y_2 - 3.4 Z_2 X_1 \mid \psi \rangle\\
= 1.2\langle \psi \mid X_0 Z_2 \mid \psi \rangle + 2.5 \langle \psi \mid Z_0 X_1 Y_2\mid \psi \rangle - 3.4 \langle \psi \mid Z_2 X_1 \mid \psi \rangle
$$

## Hamiltonian expectations and sampling
The expected value of the Hamiltonian can be obtained from the sampling of the calculation results. The expected value for the Hamiltonian $H=Z$ is as follows.

$$
\langle \psi \mid Z \mid \psi \rangle = 
\begin{bmatrix}
\alpha^* & \beta^*
\end{bmatrix}
\begin{bmatrix}
1&0\\
0&-1
\end{bmatrix}
\begin{bmatrix}
\alpha\\
\beta
\end{bmatrix}
= |\alpha|^2 - |\beta|^2
$$

$|alpha|^2$ and $|\beta|^2$ are the probabilities of 0 and 1, respectively. Perform multiple calculations and find the expected value from that sample value.

Normally, when the Hamiltonian is X or Y, it is not possible to find the expected value from the sample value. In this case, the rotation of each axis is used to adjust the sample so that it can be taken.

For $X$ we use $X = HZH$

$$
\langle \psi \mid X \mid \psi \rangle \\
= \langle \psi \mid HZH \mid \psi \rangle\\
= \langle \psi' \mid Z \mid \psi' \rangle
$$

For $Y$ we use $Y = RX(-\pi/2) Z RX(\pi/2)$

$$
\langle \psi \mid Y \mid \psi \rangle \\
= \langle \psi \mid RX(-\pi/2) Z RX(\pi/2) \mid \psi \rangle\\
= \langle \psi'' \mid Z \mid \psi'' \rangle
$$

In this case, the corresponding rotating gate is inserted just before the measurement.

## Quantum Variational Principle
In an arbitrary state vector $\psi(\theta)$, the Hamiltonian expectation satisfies the following

$$
\langle \psi (\theta) \mid H \mid \psi (\theta) \rangle \geq E_0
$$

VQE takes advantage of this quantum variational principle and uses an optimization algorithm from an existing computer to find a minimum value that is as close to $E_0$ as possible while changing the state vector with the angle parameter $\theta$.


## Ansatz
The quantum circuit for efficiently finding the minimum value is called Ansatz. Currently, efficient quantum circuits such as UCC Ansatz for quantum chemistry and QAOA Ansatz for combinatorial optimization problems have been found and are expected to be an application. Ansatz has rules for each field to some extent and is written according to the rules.

## Example.
Finally, let's do an example.

1. Create an Ansatz with the angle of the rotating gate as a parameter. (Quantum)
2. Compute the following from the results of the run, $\langle \psi (\theta) \mid H \mid \psi (\theta) \rangle$ (Classical)
3. Try the next suggested angle parameters from the classical optimizer.

This time, as the Ansatz, use an arbitrary one.

```
rx(a)[0].rz(b)[0]
```

I would like to calculate one qubit using two angles, a and b. For the Hamiltonian, I'll use the example that came up above. Finally, let's compare the results of VQE's calculations with those of the numerical library numpy.

In [2]:
import numpy as np
from blueqat import Circuit
from blueqat.pauli import X, Y, Z, I
from blueqat.vqe import AnsatzBase, Vqe

class OneQubitAnsatz(AnsatzBase):
    def __init__(self, hamiltonian):
        super().__init__(hamiltonian.to_expr(), 2)
        self.step = 1

    def get_circuit(self, params):
        a, b = params
        return Circuit().rx(a)[0].rz(b)[0]

# hamiltonian
h = 1.23 * I - 4.56 * X(0) + 2.45 * Y(0) + 2.34 * Z(0)

runner = Vqe(OneQubitAnsatz(h))
result = runner.run()

print('Result by VQE')
print(runner.ansatz.get_energy_sparse(result.circuit))

# This is for check
mat = h.to_matrix()
print('Result by numpy')
print(np.linalg.eigh(mat)[0][0])

Result by VQE
-4.4478075472540315
Result by numpy
-4.450818602983201


Now you did VQE

# Set up a classical optimization solver

Here we see the classical optimization solver used in VQE or QAOA quantum-classical hybrid algorithms.
When we calculate the ansatz we just set up the minimizer.

In [1]:
import numpy as np
from blueqat import Circuit
from blueqat.pauli import X, Y, Z, I
from blueqat.pauli import qubo_bit as q
from blueqat.vqe import AnsatzBase, Vqe, get_scipy_minimizer

class QubitAnsatz(AnsatzBase):
    def __init__(self, hamiltonian):
        super().__init__(hamiltonian, 4)
        self.step = 1

    def get_circuit(self, params):
        a, b, c, d = params
        return Circuit().ry(a)[0].rz(b)[0].ry(c)[1].rz(d)[1]

h = -3*q(0)-3*q(1)-2*q(0)*q(1)
h = h.to_expr().simplify()
minimizer = get_scipy_minimizer(method="COBYLA",options={"tol":5.0e-4})
runner = Vqe(QubitAnsatz(h),minimizer=minimizer)
result = runner.run()

print('Result by VQE')
print(runner.ansatz.get_energy_sparse(result.circuit))

# Hamiltonian to matrix
mat = h.to_matrix()

# Calculate by numpy
print('Result by numpy')
print(np.linalg.eigh(mat)[0][0])

Result by VQE
-7.999999852187695
Result by numpy
-8.0


## Other solvers
If you want to use your own solver, it can be set up by making a function. This time we try to use bayesian optimization solver "hyperopt".

In [ ]:
!pip install hyperopt

In [3]:
def hyperopt_minimizer(objective, n_params):
    from hyperopt import fmin, Trials, tpe, hp
    trials = Trials()
    best = fmin(objective, [hp.uniform(f'p{i}', 0., 2 * np.pi) for i in range(n_params)],
            algo=tpe.suggest, max_evals=100, trials=trials, verbose=1)
    return list(best.values())

In [4]:
runner = Vqe(QubitAnsatz(h),minimizer=hyperopt_minimizer)
result = runner.run()

print('Result by VQE')
print(runner.ansatz.get_energy_sparse(result.circuit))

# Hamiltonian to matrix
mat = h.to_matrix()

# Calculate by numpy
print('Result by numpy')
print(np.linalg.eigh(mat)[0][0])

100%|██████████| 100/100 [00:00<00:00, 119.32trial/s, best loss: -7.977775163020699]
Result by VQE
-7.977775163020699
Result by numpy
-8.0


We succeded to set up an optimizer.